In [1]:
#preamble
import os
from math import floor

In [2]:
work_dir = "/media/geoff/datasets/soapies_balanced_corpora/"
transcriptions_file = work_dir + "soapiescorpus_transcription_status_20170710.txt"

In [3]:
os.path.isdir(work_dir)

True

### Create dictionary for the episode num, speaker name, and utterance number and the utterance ids

In [4]:
speak_to_id = {}
id_to_speak = {}

for lang_pair in ["zul","xho","sot","tsn"]:
    # speakers_to_id_file = work_dir + "cs_eng" + lang_pair + "_balanced/linksmaps_ids.no_s.txt"
    speakers_to_id_file = work_dir + "cs_eng" + lang_pair + "_balanced/transcriptions/linksmaps_ids.txt"
    with open(speakers_to_id_file) as f:
        for line in f:
            #speaker = line.split("_")[0]
            #ep_id = line.split("_")[1]
            #utt_num = line.split()[0].split("_")[2]
            utt_id_from_episode_info = line.split()[0]
            
            utt_id = line.split()[1].rstrip()
            
            speak_to_id[(utt_id_from_episode_info)] = (utt_id,lang_pair)
            id_to_speak[(utt_id,lang_pair)] = (utt_id_from_episode_info)

In [5]:
len(speak_to_id)

54393

### List of all utterances with language and transcriptions

In [6]:
trancriptions_w_utt_inf = {}

with open(os.path.join(work_dir,transcriptions_file)) as f:
    for line in f:
        if line[0] == "#":
            pass
        else:
            ep_id = "_".join(line.split()[0].split("_")[1:])
            speaker = line.split()[1]
            utt_num = line.split()[2]
            seg_stats = line.split()[9:]
            
            utt_id_from_episode_info = speaker + "_" + ep_id + "_" + utt_num
            
            #print(ep_id)
            #print(speaker)
            #print(utt_num)
            #print(seg_stats)
            trancriptions_w_utt_inf[utt_id_from_episode_info] = seg_stats

In [7]:
trancriptions_w_utt_inf['STEVE_100_1']

['705.0',
 '11696.0',
 '12401.0',
 'Setswana',
 'ah#o#sa#ntso#u#na#le#di',
 '907.0',
 '12401.0',
 '13308.0',
 'English',
 'feelings#for#Lungile']

In [8]:
print(trancriptions_w_utt_inf[id_to_speak[('s00004_u00327','zul')]])
print(id_to_speak[('s00004_u00327','zul')])
#14-01-06 MAMPHO 24

['312.666666667', '438736.0', '439048.666667', 'isiZulu', 'uqale#nge-', '312.666666667', '439048.666667', '439361.333333', 'English', 'toilet', '312.666666667', '439361.333333', '439674.0', 'isiZulu', 'ntwana']
BASH_13-08-02_105


In [9]:
trancriptions_w_utt_inf[id_to_speak[('s00001_u00836','zul')]]

['386.0',
 '633416.0',
 '633802.0',
 'English',
 'at#least',
 '598.0',
 '633802.0',
 '634400.0',
 'isiZulu',
 'zamanyana']

In [10]:
len(trancriptions_w_utt_inf)

80841

### Create files of utt lang information for the balanced corpora

In [11]:
multi_targ = True

for lang_pair in ["zul","xho","sot","tsn"]:#,,"sot"]: #zul
    audio_root_folder = work_dir + "cs_eng" + lang_pair + "_balanced"
    #audio_list_file = os.path.join(audio_root_folder,"lists_unbalanced")
    audio_list_file = os.path.join(audio_root_folder,"lists")
    if multi_targ:
        audio_lang_targ_file = os.path.join(audio_root_folder,"lang_targs_mult")
    else:
        audio_lang_targ_file = os.path.join(audio_root_folder,"lang_targs")
    
    if not os.path.isdir(audio_lang_targ_file):
        os.mkdir(audio_lang_targ_file)
        
    for set in ["trn","dev","tst"]:
        print(lang_pair)
        print(set)
        with open(os.path.join(audio_list_file,set + ".lst")) as f:
            for line in f: 
                utt_lang_target = []
                
                # get ids in terms of speaker and episode
                utt_id = ('s'+line.split("/s")[1].split(".")[0],lang_pair)
                speak_id = id_to_speak[utt_id]
                #print(utt_id, " -> ", speak_id)
                
                # get utterance information
                if speak_id in list(trancriptions_w_utt_inf.keys()): # Would appear I have some transcriptions missing
                    seg_stats = trancriptions_w_utt_inf[speak_id]
                    segs = len(seg_stats) // 5
                    frames_total = 0
                    frames_prev = 0
                    #print(seg_stats)
                    for i in range(segs):
                        frames_total += float(seg_stats[i*5])*16
                        frames_new = round(frames_total) - round(frames_prev) # dur*f_s/1000 = frames
                        lang = seg_stats[3+i*5]
                        
                        #if not frames.is_integer():
                        #    raise Exception("Error non integer number frames, got:",frames)
                        if multi_targ:
                            if seg_stats[3+i*5] == "English":
                                for j in range(frames_new): 
                                    utt_lang_target.append(1)
                            elif seg_stats[3+i*5] == "isiZulu":
                                for j in range(frames_new):
                                    utt_lang_target.append(2)
                            elif seg_stats[3+i*5] == "isiXhosa":
                                for j in range(frames_new):
                                    utt_lang_target.append(3)
                            elif seg_stats[3+i*5] == "Sesotho":
                                for j in range(frames_new):
                                    utt_lang_target.append(4)
                            elif seg_stats[3+i*5] == "Setswana":
                                for j in range(frames_new):
                                    utt_lang_target.append(5)
                        else:
                            if seg_stats[3+i*5] == "English":
                                for j in range(frames_new): 
                                    utt_lang_target.append(1)
                            else:
                                for j in range(frames_new):
                                    utt_lang_target.append(2)

                        frames_prev = frames_total
                                
                    #print(len(utt_lang_target))
                    
                    # write utt lang targs to file
                    audio_lang_targ_speaker_file = os.path.join(audio_lang_targ_file,utt_id[0].split("_")[0])
                    
                    if not os.path.isdir(audio_lang_targ_speaker_file):
                        os.mkdir(audio_lang_targ_speaker_file)
                        
                    with open(os.path.join(audio_lang_targ_speaker_file,(utt_id[0] + ".txt")),"w") as f:
                        joined = "".join(str(x) for x in utt_lang_target)
                            
                        f.write(joined)

zul
trn
zul
dev
zul
tst
xho
trn
xho
dev
xho
tst
sot
trn
sot
dev
sot
tst
tsn
trn
tsn
dev
tsn
tst
